# Entity Service Permutation Output

This tutorial demonstrates the workflow for private record linkage using the entity service. Two parties _Alice_ and _Bob_ have a dataset of personally identifiable information (PII) of several entities. They want to learn the linkage of corresponding entities between their respective datasets with the help of the entity service and an independent party, the _Analyst_.

The chosen output type is `permuatations`, which consists of two permutations and one mask.


### Who learns what?

After the linkage has been carried out Alice and Bob will be able to retrieve a `permutation` - a reordering of their respective data sets such that shared entities line up.

The Analyst - who creates the linkage project - learns the `mask`. The mask is a binary vector that indicates which rows in the permuted data sets are aligned. Note this reveals how many entities are shared.


### Steps
These steps are usually run by different companies - but for illustration all is carried out in this one file. The participants providing data are _Alice_ and *Bob*, and the *Analyst* acting the integration authority.

* [Check connection to Entity Service](#check_con)
* [Data preparation](#data_prep)
  * Write CSV files with PII
  * [Create a Linkage Schema](#schema_prep)
* [Create Linkage Project](#create_pro)
* [Generate CLKs from PII](#hash_n_up)
* [Upload the PII](#hash_n_up)
* [Create a run](#create_run)
* [Retrieve and analyse results](#results)

<a id="check_con"></a>
## Check Connection

If you are connecting to a custom entity service, change the address here.

In [1]:
url = 'https://testing.es.data61.xyz'

In [2]:
!clkutil status --server "{url}"

{"project_count": 2250, "rate": 96222330, "status": "ok"}


<a id="data_prep"></a>
## Data preparation

Following the [clkhash tutorial](http://clkhash.readthedocs.io/en/latest/tutorial_cli.html) we will use a dataset from the `recordlinkage` library. We will just write both datasets out to temporary CSV files.

If you are following along yourself you may have to adjust the file names in all the `!clkutil` commands.

In [3]:
from tempfile import NamedTemporaryFile
from recordlinkage.datasets import load_febrl4

In [4]:
dfA, dfB = load_febrl4()

a_csv = NamedTemporaryFile('w')
a_clks = NamedTemporaryFile('w', suffix='.json')
dfA.to_csv(a_csv)
a_csv.seek(0)

b_csv = NamedTemporaryFile('w')
b_clks = NamedTemporaryFile('w', suffix='.json')
dfB.to_csv(b_csv)
b_csv.seek(0)

dfA.head()
print("Datasets written to {} and {}".format(a_csv.name, b_csv.name))

Datasets written to /tmp/tmps3th_dx6 and /tmp/tmp46b15qa0


In [5]:
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


<a id="schema_prep"></a>
## Schema Preparation

The linkage schema must be agreed on by the two parties. A hashing schema instructs clkhash how to treat each column for generating CLKs. A detailed description of the hashing schema can be found in the [api docs](http://clkhash.readthedocs.io/en/latest/schema.html). We will ignore the columns ‘rec_id’ and ‘soc_sec_id’ for CLK generation.

In [6]:
schema = NamedTemporaryFile('wt')

In [7]:
%%writefile {schema.name}
{
  "version": 1,
  "clkConfig": {
    "l": 1024,
    "k": 30,
    "hash": {
      "type": "doubleHash"
    },
    "kdf": {
      "type": "HKDF",
      "hash": "SHA256",
        "info": "c2NoZW1hX2V4YW1wbGU=",
        "salt": "SCbL2zHNnmsckfzchsNkZY9XoHk96P/G5nUBrM7ybymlEFsMV6PAeDZCNp3rfNUPCtLDMOGQHG4pCQpfhiHCyA==",
        "keySize": 64
    }
  },
  "features": [
    {
      "identifier": "rec_id",
      "ignored": true
    },
    {
      "identifier": "given_name",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 1 }
    },
    {
      "identifier": "surname",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 1 }
    },
    {
      "identifier": "street_number",
      "format": { "type": "integer" },
      "hashing": { "ngram": 1, "positional": true, "weight": 0.5, "missingValue": {"sentinel": ""} }
    },
    {
      "identifier": "address_1",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 0.5 }
    },
    {
      "identifier": "address_2",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 0.5 }
    },
    {
      "identifier": "suburb",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 0.5 }
    },
    {
      "identifier": "postcode",
      "format": { "type": "integer", "minimum": 100, "maximum": 9999 },
      "hashing": { "ngram": 1, "positional": true, "weight": 0.5 }
    },
    {
      "identifier": "state",
      "format": { "type": "string", "encoding": "utf-8", "maxLength": 3 },
      "hashing": { "ngram": 2, "weight": 1 }
    },
    {
      "identifier": "date_of_birth",
      "format": { "type": "integer" },
      "hashing": { "ngram": 1, "positional": true, "weight": 1, "missingValue": {"sentinel": ""} }
    },
    {
      "identifier": "soc_sec_id",
      "ignored": true
    }
  ]
}

Overwriting /tmp/tmpfr9cs71y


<a id="create_pro"></a>
## Create Linkage Project

The analyst carrying out the linkage starts by creating a linkage project of the desired output type with the Entity Service.


In [8]:
creds = NamedTemporaryFile('wt')
print("Credentials will be saved in", creds.name)

!clkutil create-project --schema "{schema.name}" --output "{creds.name}" --type "permutations" --server "{url}"
creds.seek(0)

import json
with open(creds.name, 'r') as f:
    credentials = json.load(f)

project_id = credentials['project_id']
credentials

Credentials will be saved in /tmp/tmpyon7ygi_
Project created


{'project_id': '962de43b0527963eb8fd1fd041bc25f56e4eea28794dbf62',
 'result_token': '4e13d802195c55ca8ff180e9cb179b6031a3fa35b4d12e3a',
 'update_tokens': ['1a264cad189606a3c37e1ed1f30aeaf45cb0b110506ea57a',
  '8906da6506dfd53a36e367286f94e8fb0664e2892ea64167']}

**Note:** the analyst will need to pass on the `project_id` (the id of the linkage project) and one of the two `update_tokens` to each data provider.

<a id="hash_n_up"></a>
## Hash and Upload

At the moment both data providers have *raw* personally identiy information. We first have to generate CLKs from the raw entity information. We need:
- the *clkhash* library
- the linkage schema from above
- and two secret passwords which are only known to Alice and Bob. (here: `horse` and `staple`)

Please see [clkhash](https://clkhash.readthedocs.io/) documentation for further details on this.

In [9]:
!clkutil hash "{a_csv.name}" horse staple "{schema.name}" "{a_clks.name}"
!clkutil hash "{b_csv.name}" horse staple "{schema.name}" "{b_clks.name}"

generating CLKs: 100%|█| 5.00k/5.00k [00:02<00:00, 602clk/s, mean=765, std=37.1]
CLK data written to /tmp/tmpssfowykz.json
generating CLKs: 100%|█| 5.00k/5.00k [00:02<00:00, 578clk/s, mean=756, std=43.3]
CLK data written to /tmp/tmp9jo0u09p.json


Now the two clients can upload their data providing the appropriate *upload tokens* and the *project_id*. As with all commands in `clkhash` we can output help:

In [10]:
!clkutil upload --help

Usage: clkutil upload [OPTIONS] INPUT

  Upload CLK data to entity matching server.

  Given a json file containing hashed clk data as INPUT, upload to the
  entity resolution service.

  Use "-" to read from stdin.

Options:
  --project TEXT         Project identifier
  --apikey TEXT          Authentication API key for the server.
  --server TEXT          Server address including protocol
  -o, --output FILENAME
  -v, --verbose          Script is more talkative
  --help                 Show this message and exit.


### Alice uploads her data

In [11]:
with NamedTemporaryFile('wt') as f:
    !clkutil upload \
        --project="{project_id}" \
        --apikey="{credentials['update_tokens'][0]}" \
        --server "{url}" \
        --output "{f.name}" \
        "{a_clks.name}"
    res = json.load(open(f.name))
    alice_receipt_token = res['receipt_token']

Every upload gets a receipt token. This token is required to access the results.

### Bob uploads his data

In [12]:
with NamedTemporaryFile('wt') as f:
    !clkutil upload \
        --project="{project_id}" \
        --apikey="{credentials['update_tokens'][1]}" \
        --server "{url}" \
        --output "{f.name}" \
        "{b_clks.name}"
    
    bob_receipt_token = json.load(open(f.name))['receipt_token']

<a id="create_run"></a>
## Create a run

Now the project has been created and the CLK data has been uploaded we can carry out some privacy preserving record linkage. Try with a few different threshold values:

In [13]:
with NamedTemporaryFile('wt') as f:
    !clkutil create \
        --project="{project_id}" \
        --apikey="{credentials['result_token']}" \
        --server "{url}" \
        --threshold 0.85 \
        --output "{f.name}"
    
    run_id = json.load(open(f.name))['run_id']

<a id="results"></a>
## Results

Now after some delay (depending on the size) we can fetch the mask.
This can be done with clkutil:

    !clkutil results --server "{url}" \
        --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" --output results.txt
        
However for this tutorial we are going to use the Python `requests` library:

In [14]:
import requests
import clkhash.rest_client
import json
import time
from IPython.display import display, clear_output

In [15]:
for update in clkhash.rest_client.watch_run_status(url, project_id, run_id, credentials['result_token'], timeout=300):
    clear_output(wait=True)
    print(clkhash.rest_client.format_run_status(update))

State: completed
Stage (3/3): compute output


In [16]:
results = requests.get('{}/api/v1/projects/{}/runs/{}/result'.format(url, project_id, run_id), headers={'Authorization': credentials['result_token']}).json()

In [17]:
mask = results['mask']

This mask is a boolean array that specifies where rows of permuted data line up.

In [18]:
print(mask[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


The number of 1s in the mask will tell us how many matches were found.

In [19]:
sum([1 for m in mask if m == 1])

4856

We also use `requests` to fetch the permutations for each data provider:

In [20]:
alice_res = requests.get('{}/api/v1/projects/{}/runs/{}/result'.format(url, project_id, run_id), headers={'Authorization': alice_receipt_token}).json()
bob_res = requests.get('{}/api/v1/projects/{}/runs/{}/result'.format(url, project_id, run_id), headers={'Authorization': bob_receipt_token}).json()

Now Alice and Bob both have a new permutation - a new ordering for their data.

In [21]:
alice_permutation = alice_res['permutation']
alice_permutation[:10]

[753, 4516, 1049, 800, 3290, 4453, 644, 3217, 4024, 1991]

This permutation says the first row of Alice's data should be moved to position 308.

In [22]:
bob_permutation = bob_res['permutation']
bob_permutation[:10]

[1636, 2245, 1000, 3934, 1927, 4009, 654, 2691, 3910, 343]

In [23]:
def reorder(items, order):
    """
    Assume order is a list of new index
    """
    neworder = items.copy()
    for item, newpos in zip(items, order):
        neworder[newpos] = item
    
    return neworder

In [24]:
with open(a_csv.name, 'r') as f:
    alice_raw = f.readlines()[1:]
    alice_reordered = reorder(alice_raw, alice_permutation)

with open(b_csv.name, 'r') as f:
    bob_raw = f.readlines()[1:]
    bob_reordered = reorder(bob_raw, bob_permutation)

Now that the two data sets have been permuted, the mask reveals where the rows line up, and where they don't.

In [25]:
alice_reordered[:10]

['rec-2218-org,tyler,berryman,223,broad place,villa 3,clayton,6355,vic,19550922,2531288\n',
 'rec-4967-org,james,feast,139,goldstein crescent,kooringle park,beenleigh,4214,vic,19500714,9202596\n',
 'rec-4240-org,solomon,hannagan,10,clemenger street,brookfield,forrestfield,4216,sa,19890520,3944717\n',
 'rec-2696-org,campbell,nguyen,6,diselma place,villa 2,collinswood,4343,nsw,19630325,2861961\n',
 'rec-4147-org,bethany,webb,3,cullen street,sunning,narellan vale,3630,qld,19431219,4827546\n',
 'rec-1948-org,flynn,painter,7,thomas hart street,railway stn,seaton,2640,vic,19211124,8126727\n',
 'rec-1225-org,hayden,ballantyne,13,,nunnook,young,2077,nsw,19330812,3414771\n',
 'rec-2896-org,blake,de courcey,26,ellerston avenue,rosehill,bowral,3090,qld,19890504,1219082\n',
 'rec-2852-org,joshua,huxley,12,southern close,littledale,maryborough,2517,nsw,19670618,1728316\n',
 'rec-438-org,alison,hearn,4,macdonnell street,cabrini medical centre,adelaide,2720,vic,19191230,2937695\n']

In [26]:
bob_reordered[:10]

['rec-2218-dup-0,tylee,berryman,223,broad place,villai3,clayton,6355,vic,19901803,2531288\n',
 'rec-4967-dup-0,jmaes,feast,139,goldstein crescent,kooringl epark,beenleogh,4214,vic,19500714,9202596\n',
 'rec-4240-dup-0,solomon,hannagan,83,clemenger street,brookfield,forrestfield,4216,sa,19890520,3944717\n',
 'rec-2696-dup-0,jenna,nguyen,85,diselmaplace,villz2,collinswood,4343,nsw,19630325,2861961\n',
 'rec-4147-dup-0,bethany,godfrey,3,culleb street,sunning,narellan vale,3603,qld,19330508,4827546\n',
 'rec-1948-dup-0,flnyn,painter,7,thomas hart street,railway stn,sea ton,2640,vic,19211124,8126727\n',
 'rec-1225-dup-0,hayden,ballantyne,13,,,young,2077,nsw,19330812,3414771\n',
 'rec-2896-dup-0,blake,mcvicar,26,ellerston avenue,rosehill,bowral,3009,qld,19890504,1219082\n',
 'rec-2852-dup-0,kiana,huxley,12,southern close,little dale,maryborough,2517,nsw,19140529,1728316\n',
 'rec-438-dup-0,heatn,alison,4,macdonnell street,cabrini medicalb centre,adelaide,2270,vic,19191230,2937695\n']

## Accuracy

To compute how well the matching went we will use the first index as our reference.

For example in `rec-1396-org` is the original record which has a match in `rec-1396-dup-0`. To satisfy ourselves we can preview the first few supposed matches:

In [27]:
for i, m in enumerate(mask[:10]):
    if m:
        entity_a = alice_reordered[i].split(',')
        entity_b = bob_reordered[i].split(',')
        name_a = ' '.join(entity_a[1:3]).title()
        name_b = ' '.join(entity_b[1:3]).title()
        
        print("{} ({})".format(name_a, entity_a[0]), '=?', "{} ({})".format(name_b, entity_b[0]))

Tyler Berryman (rec-2218-org) =? Tylee Berryman (rec-2218-dup-0)
James Feast (rec-4967-org) =? Jmaes Feast (rec-4967-dup-0)
Solomon Hannagan (rec-4240-org) =? Solomon Hannagan (rec-4240-dup-0)
Campbell Nguyen (rec-2696-org) =? Jenna Nguyen (rec-2696-dup-0)
Bethany Webb (rec-4147-org) =? Bethany Godfrey (rec-4147-dup-0)
Flynn Painter (rec-1948-org) =? Flnyn Painter (rec-1948-dup-0)
Hayden Ballantyne (rec-1225-org) =? Hayden Ballantyne (rec-1225-dup-0)
Blake De Courcey (rec-2896-org) =? Blake Mcvicar (rec-2896-dup-0)
Joshua Huxley (rec-2852-org) =? Kiana Huxley (rec-2852-dup-0)
Alison Hearn (rec-438-org) =? Heatn Alison (rec-438-dup-0)


### Metrics
If you know the ground truth — the correct mapping between the two datasets — you can compute performance metrics of the linkage.

**Precision**: The percentage of actual matches out of all found matches. (`tp/(tp+fp)`)

**Recall**: How many of the actual matches have we found? (`tp/(tp+fn)`)

In [28]:
tp = 0
fp = 0

for i, m in enumerate(mask):
    if m:
        entity_a = alice_reordered[i].split(',')
        entity_b = bob_reordered[i].split(',')
        if entity_a[0].split('-')[1] == entity_b[0].split('-')[1]:
            tp += 1
        else:
            fp += 1
            #print('False positive:',' '.join(entity_a[1:3]).title(), '?', ' '.join(entity_b[1:3]).title(), entity_a[-1] == entity_b[-1])

print("Found {} correct matches out of 5000. Incorrectly linked {} matches.".format(tp, fp))
precision = tp/(tp+fp)
recall = tp/5000

print("Precision: {:.1f}%".format(100*precision))
print("Recall: {:.1f}%".format(100*recall))

Found 4846 correct matches out of 5000. Incorrectly linked 10 matches.
Precision: 99.8%
Recall: 96.9%
